In [1]:
import pandas as pd
from collections import Counter

In [2]:
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
from catboost import CatBoostClassifier, Pool

In [4]:
def get_mode(row): # Кастомная функция вычисления модального значения категориального признака
    counter = Counter(row)
    (most_common, _) = counter.most_common(1)[0]
    
    return most_common

## Загрузка таблиц

Загрузим тестовый файл и файл с оописанием видео для генерации фич

In [6]:
path = 'test_dataset_Тест/' # тест
path_ = 'train_dataset_soc_dem_train/' #данные о видео
data = pd.read_csv(path + 'test_events.csv')
video = pd.read_csv(path_ + 'video_info.csv')
#targets = pd.read_csv(path + 'train_targets.csv')

In [7]:
data.head(2)

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531


In [8]:
video.head(2)

,rutube_video_id,title,category,duration,author_id
0,video_185549,Как собрать букет из мыльных тюльпанов - Силик...,Хобби,1559160,1015054
1,video_111035,"Осторожно, Киберземляне!, 1 сезон, 12 серия",Сериалы,1320007,1002180


## Генерация фич для тестового датасета

Приведем время к местному времени

In [9]:
diff_msk = {'Kaliningrad Oblast': -1, 'Kaliningrad': -1,
'Komi':0, 'Karelia':0, 'Arkhangelskaya': 0, 'Arkhangelsk Oblast': 0, 'Nenets':0, 'Vologda Oblast':0, 'Vologda':0,    
'St.-Petersburg': 0,  "Leningradskaya Oblast'":0, 'Murmansk':0, 'Novgorod Oblast':0,  'Pskov Oblast':0,
'Moscow': 0, 'Moscow Oblast': 0, 'Belgorod Oblast': 0, 'Bryansk Oblast': 0, 'Vladimir':0,  'Vladimir Oblast': 0, 
'Voronezh Oblast':0, 'Voronezj':0,  'Ivanovo Oblast':0, 'Ivanovo':0, 'Kaluga Oblast':0, 'Kaluga':0, 
'Kostroma Oblast':0,  'Kursk':0, 'Kursk Oblast': 0, 'Lipetsk Oblast':0, 'Oryol oblast':0, 'Orel Oblast':0, 
'Ryazan Oblast': 0, 'Smolensk':0,'Smolenskaya Oblast’':0, 'Smolensk Oblast': 0, 'Tambov': 0, 'Tambov Oblast': 0,
'Tver’ Oblast':0, 'Tver Oblast': 0, 'Tula Oblast':0, 'Tula':0, 'Jaroslavl': 0, 'Yaroslavl Oblast': 0, 
'Kirov':0, 'Kirov Oblast': 0, 'Nizhny Novgorod Oblast':0, 'Penza Oblast':0, 'Penza':0, 'Mariy-El Republic': 0, 
'Tatarstan Republic': 0, 'Chuvashia': 0, 'Mordoviya Republic':0, 
'Adygeya Republic': 0, 'Volgograd Oblast':0, 'Kalmykiya Republic': 0, 'Krasnodarskiy': 0,  'Krasnodar Krai': 0,
'Crimea': 0, 'Rostov': 0, 'Sebastopol City': 0,
'Dagestan': 0, 'Ingushetiya Republic': 0, 'Kabardino-Balkariya Republic': 0, 'Karachayevo-Cherkesiya Republic': 0,
'North Ossetia': 0,  'North Ossetia–Alania': 0, 'Stavropol Kray': 0, 'Stavropol’ Kray': 0, 'Stavropol Krai': 0,
'Chechnya': 0, 
'Astrakhan': 1, 'Astrakhan Oblast':1, 'Samara Oblast': 1, 'Saratov Oblast': 1,  'Saratovskaya Oblast': 1, 
'Ulyanovsk': 1, 'Udmurtiya Republic': 1, 
'Chelyabinsk': 2, 'Bashkortostan Republic': 2, 'Perm Krai': 2, 'Perm': 2, 'Kurgan Oblast': 2, 
'Sverdlovsk': 2, 'Sverdlovsk Oblast': 2, 'Tyumen Oblast': 2, 'Tyumen’ Oblast': 2, 'Orenburg Oblast': 2, 
'Yamalo-Nenets': 2, 'Khanty-Mansia': 2,
'Omsk Oblast': 3,  'Omsk': 3,  
'Altay Kray': 4, 'Altai': 4, 'Krasnoyarsk Krai': 4, 'Krasnoyarskiy': 4,  'Novosibirsk Oblast': 4, 
'Kuzbass':4, 'Kemerovo Oblast': 4, 'Khakasiya Republic': 4, 'Tyva Republic': 4, 'Tomsk Oblast': 4, 
'Irkutsk Oblast': 5, 'Transbaikal Territory':5, 'Zabaykalskiy (Transbaikal) Kray': 5, 'Buryatiya Republic':5,
'Amur Oblast': 6, 'Sakha': 6,                             
'Primorye': 7, 'Primorskiy (Maritime) Kray': 7, 'Jewish Autonomous Oblast': 7, 'Khabarovsk': 7,
'Sakhalin Oblast': 8,  'Magadan Oblast': 8,
'Kamchatka': 9, 'Chukotka': 9, }

In [10]:
#время к местному
data['event_timestamp']=pd.to_datetime(data['event_timestamp'])
data['diff_msk']=data['region'].map(diff_msk)
data['diff_msk']=pd.to_timedelta(data['diff_msk'], unit='hours')
data['timestamp']=data['event_timestamp'].dt.tz_localize(None)+data['diff_msk']

#понедельник 1... вск 7
data['weekday'] = data['timestamp'].dt.weekday + 1
data['hour'] = data['timestamp'].dt.hour

In [11]:
data.head(2)

,event_timestamp,region,ua_device_type,ua_client_type,ua_os,ua_client_name,total_watchtime,rutube_video_id,viewer_uid,diff_msk,timestamp,weekday,hour
0,2024-06-01 15:26:44+03:00,Udmurtiya Republic,desktop,browser,Windows,Yandex Browser,2593,video_212730,22206,0 days 01:00:00,2024-06-01 16:26:44,6,16
1,2024-06-01 12:25:29+03:00,Adygeya Republic,smartphone,mobile app,Android,Rutube,960,video_235114,34531,0 days 00:00:00,2024-06-01 12:25:29,6,12


In [12]:
# 'weekday_hour'
data_time = pd.get_dummies(data[['weekday', 'hour', 'viewer_uid']], 
                           columns=['weekday', 'hour']).groupby('viewer_uid', as_index=False).sum()
data_time.head(2)

,viewer_uid,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,hour_0,hour_1,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,0,1,2,2,1,0,2,1,2,1,...,0,2,0,1,0,0,0,0,2,1
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Основные характеристики тестовых пользователей

In [13]:
data_user = data.groupby('viewer_uid', as_index=False).agg(
    watching=('viewer_uid', 'count'), 
    region=('region', get_mode),
    device_type=('ua_device_type', get_mode),
    client_type=('ua_client_type', get_mode),
    os=('ua_os', get_mode),
    client_name=('ua_client_name', get_mode),
)

In [14]:
data_user.head(2)

,viewer_uid,watching,region,device_type,client_type,os,client_name
0,0,9,Oryol oblast,smartphone,mobile app,NaN,Rutube
1,1,1,Mariy-El Republic,desktop,browser,Windows,Yandex Browser


Фичи по видео

In [15]:
merged_df = pd.merge(data[['total_watchtime', 'rutube_video_id', 'viewer_uid']], video, on=['rutube_video_id'])
merged_df['duration']=merged_df['duration']/1000 #в секунды
merged_df['watch_part']=(100*merged_df['total_watchtime']/merged_df['duration']).round(1) #процент просмотра
merged_df.head(2)

,total_watchtime,rutube_video_id,viewer_uid,title,category,duration,author_id,watch_part
0,2593,video_212730,22206,Отчаянные Домохозяйки 7 сезон 21 серия,Сериалы,2610.785,1089828,99.3
1,2603,video_212730,13237,Отчаянные Домохозяйки 7 сезон 21 серия,Сериалы,2610.785,1089828,99.7


In [16]:
merged_video = merged_df.groupby('viewer_uid', as_index=False).agg(
    video_count=('rutube_video_id', 'count'), #кол-во просмотренных видео
    video_unique=('rutube_video_id', 'nunique'), #кол-во уникальных видео
    watch_min=('total_watchtime', 'min'),     #по времени
    watch_max=('total_watchtime', 'max'),
    watch_median=('total_watchtime', 'median'),
    watch_mean=('total_watchtime', 'mean'),
    part_min=('watch_part', 'min'),           #по проценту просмотра
    part_max=('watch_part', 'max'),
    part_median=('watch_part', 'median'),
    part_mean=('watch_part', 'mean'),
    category=('category', get_mode),
    author_id=('author_id', get_mode),
).round(1)

merged_video.head(2)

,viewer_uid,video_count,video_unique,watch_min,watch_max,watch_median,watch_mean,part_min,part_max,part_median,part_mean,category,author_id
0,0,9,9,60,4821,1542.0,1602.1,2.1,99.7,35.7,48.2,Интервью,1010000
1,1,1,1,92,92,92.0,92.0,5.7,5.7,5.7,5.7,Интервью,1011365


In [17]:
data_video = pd.get_dummies(merged_df[['category', 'viewer_uid']], columns=['category',]).groupby('viewer_uid', as_index=False).sum()

In [18]:
data_video.head(2)

,viewer_uid,category_Авто-мото,category_Аниме,category_Аудио,category_Аудиокниги,category_Бизнес и предпринимательство,category_Видеоигры,category_Детям,category_Дизайн,category_Еда,...,category_Сериалы,category_Спорт,category_Строительство и ремонт,category_Телепередачи,category_Техника и оборудование,category_Технологии и интернет,category_Фильмы,category_Хобби,category_Эзотерика,category_Юмор
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Объдиним все фичи в один датфрейм

In [19]:
all_df = pd.merge(data_time, data_user, on=['viewer_uid'])
all_df = pd.merge(all_df, merged_video, on=['viewer_uid'])
all_df = pd.merge(all_df, data_video, on=['viewer_uid'])
#all_df = pd.merge(all_df, targets, on=['viewer_uid'])
#заполнение пропусков
all_df['os']=all_df['os'].fillna("_")
all_df

,viewer_uid,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,hour_0,hour_1,...,category_Сериалы,category_Спорт,category_Строительство и ремонт,category_Телепередачи,category_Техника и оборудование,category_Технологии и интернет,category_Фильмы,category_Хобби,category_Эзотерика,category_Юмор
0,0,1,2,2,1,0,2,1,2,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,4,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,7,3,5,6,12,7,12,0,0,...,50,0,0,0,0,0,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59999,59999,0,0,0,0,0,0,5,0,0,...,0,0,0,4,0,0,1,0,0,0
60000,60000,0,0,0,0,0,0,2,0,0,...,0,0,0,1,0,0,0,0,0,0
60001,60001,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
60002,60002,0,0,0,0,0,0,2,0,0,...,0,0,0,2,0,0,0,0,0,0


In [20]:
num_col = ['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4','weekday_5', 'weekday_6', 'weekday_7', 'hour_0', 
           'hour_1', 'hour_2','hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 
           'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 
           'hour_20', 'hour_21', 'hour_22', 'hour_23', 'watching', 'video_unique', 'watch_min', 'watch_max',
           'watch_median', 'watch_mean', 'part_min', 'part_max', 'part_median', 'part_mean', 
           'category_Авто-мото', 'category_Аниме', 'category_Аудио', 'category_Аудиокниги', 
           'category_Бизнес и предпринимательство', 'category_Видеоигры', 'category_Детям', 'category_Дизайн',
           'category_Еда', 'category_Животные', 'category_Здоровье',
           'category_Интервью', 'category_Красота', 'category_Культура',
           'category_Лайфстайл', 'category_Лайфхаки', 'category_Люди и блоги',
           'category_Музыка', 'category_Мультфильмы', 'category_Наука',
           'category_Недвижимость', 'category_Обзоры и распаковки товаров',
           'category_Обучение', 'category_Охота и рыбалка', 'category_Природа',
           'category_Психология', 'category_Путешествия', 'category_Развлечения',
           'category_Разное', 'category_Сад и огород', 'category_Сериалы',
           'category_Спорт', 'category_Строительство и ремонт',
           'category_Телепередачи', 'category_Техника и оборудование',
           'category_Технологии и интернет', 'category_Фильмы', 'category_Хобби',
           'category_Эзотерика', 'category_Юмор',]
cat_col = ['region', 'device_type', 'client_type', 'os', 'client_name', 'category', 'author_id']
#targets_col = ['viewer_uid', 'age', 'sex', 'age_class']

### Предсказание пола

In [47]:
X_test = all_df[num_col+cat_col]

In [48]:
catc = CatBoostClassifier(loss_function = "Logloss", 
                          eval_metric="F1",  
                          learning_rate=0.1, 
                          max_depth = 16, 
                          n_estimators = 2000, 
                          auto_class_weights='Balanced',
                          leaf_estimation_iterations=1, random_state=42, task_type='CPU')

In [49]:
 #для загрузки использовать
catc.load_model('model_sex.cbm')

In [50]:
sex_predict = catc.predict(X_test)

### Предсказание возраста

In [51]:
catc_ = CatBoostClassifier(loss_function = "MultiClass", 
                          eval_metric="AUC",  
                          learning_rate=0.5, 
                          max_depth = 12, 
                          n_estimators = 8000, 
                          auto_class_weights='Balanced',
                          #l2_leaf_reg = 1.5,
                          leaf_estimation_iterations=1, random_state=42, task_type='CPU')

In [52]:
  #для загрузки использовать
catc_.load_model('model_age.cbm')

In [53]:
age_predict = catc_.predict(X_test)

### Создание сабмита

In [54]:
submission = all_df.groupby('viewer_uid', as_index=False).agg(viewer_uid=('viewer_uid', get_mode))
#submission = pd.DataFrame(columns=['viewer_uid', 'sex', 'age_class'])

In [55]:
submission['viewer_uid'] = viewer_uid
submission['age'] = 35  #не знали, что это должно войти в результат, поэтому случайная цифра
submission['sex'] = sex_predict
submission['age_class'] = age_predict

In [56]:
submission.head()

,viewer_uid,age,sex,age_class
0,0,35,male,3
1,1,35,male,3
2,2,35,male,1
3,3,35,male,3
4,4,35,male,1


In [58]:
submission[['viewer_uid', 'age', 'sex', 'age_class']].to_csv('submission2.csv', index=False)